In [2]:
import pandas as pd
import sqlite3

In [3]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/sql-murder-mystery.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

In [4]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [5]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

crime_scene_report
drivers_license
person
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
income
solution


In [6]:
query = '''
SELECT * 
FROM person
'''

df = sql_query(query)
df.head(10)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
5,10027,Antione Godbolt,439509,2431,Zelham Dr,491650087
6,10034,Kyra Buen,920494,1873,Sleigh Dr,332497972
7,10039,Francesco Agundez,278151,736,Buswell Dr,861079251
8,10095,Leslie Thate,729987,2772,Camellia Park Circle,127944356
9,10122,Alva Conkel,779002,116,Diversey Circle,148521773


In [7]:
query = '''
SELECT * 
FROM crime_scene_report
'''

df = sql_query(query)
df.head(10)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
5,20180115,theft,Big Bully stole my lunch money!,Chicago
6,20180115,fraud,"Lorem ipsum dolor sit amet, consectetur adipis...",Seattle
7,20170712,theft,"A lone hunter stalks the night, firing arrows ...",SQL City
8,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it ...",SQL City
9,20171110,robbery,The Gjallarhorn shoulder-mounted rocket system...,SQL City


In [17]:
query = '''
SELECT * 
FROM crime_scene_report
WHERE city = "SQL City"
AND date = 20180115;

'''

df = sql_query(query)
df

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [19]:
df["description"][2]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [ ]:
Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave"

In [24]:
query = '''
SELECT * 
FROM person
WHERE name like "%Annabel%" or ;

'''

df = sql_query(query)
df

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143
1,78354,Annabell Siona,158932,978,Whitewater Dr,800278294
2,78799,Annabell Droneburg,984316,1944,W Natalie Dr,478793500
3,86541,Annabell Zwilling,709133,1859,Patti Rd,332961158


In [27]:
query = '''
SELECT id,name, MAX(address_number) as address_number
FROM person
WHERE address_street_name like "%Northwestern%";

'''

df = sql_query(query)
df

,id,name,address_number
0,14887,Morty Schapiro,4919


In [28]:
query = '''
SELECT id,name, MAX(address_number) as address_number
FROM person
WHERE address_street_name like "%Northwestern%";

'''

df = sql_query(query)
df

,id,name,address_number
0,14887,Morty Schapiro,4919


In [29]:


query = '''
select *
from interview
where person_id = 14887 or person_id = 16371;

'''

df = sql_query(query)
df

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


In [31]:
df["transcript"][1]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [32]:
df["transcript"][0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [33]:


query = '''
select
	person.id, person.name
from
	drivers_license
inner JOIN
	person on drivers_license.id = person.license_id,
    get_fit_now_member on person.id = get_fit_now_member.person_id,
    get_fit_now_check_in on get_fit_now_member.id = get_fit_now_check_in.membership_id
where plate_number like "%H42W%" and check_in_date = 20180109;

'''

df = sql_query(query)
df

,id,name
0,67318,Jeremy Bowers


In [34]:
query = '''
select *
from interview
where person_id = 67318;

'''

df = sql_query(query)
df

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


In [35]:
df["transcript"][0]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

In [36]:
query = '''
SELECT person.id, person.name
from drivers_license
inner join person on drivers_license.id = person.license_id
INNER join facebook_event_checkin on person.id = facebook_event_checkin.person_id
where car_model = "Model S" and hair_color = "red" and drivers_license.gender = "female";

'''

df = sql_query(query)
df

,id,name
0,99716,Miranda Priestly
1,99716,Miranda Priestly
2,99716,Miranda Priestly


In [38]:
df.name

0    Miranda Priestly
1    Miranda Priestly
2    Miranda Priestly
Name: name, dtype: object